In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
BASE_FOLDER = '/content/drive/MyDrive/SemesterProject/ML_modeling/'

In [3]:
cd '/content/drive/MyDrive/SemesterProject/ML_modeling/'

/content/drive/MyDrive/SemesterProject/ML_modeling


In [ ]:
!pip install pytorch-lightning==1.9.4
!pip install torchtext
# !pip install pandas==1.3.5
!pip install pandas
!pip install torchsampler
!pip install wandb

# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pickle
import torch
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pytorch_lightning as pl


from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import Accuracy, F1Score, ConfusionMatrix
from multiprocessing import cpu_count
from torchsampler import ImbalancedDatasetSampler

import os
from glob import glob
from multiprocessing import cpu_count

from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import wandb

In [ ]:
from datasets import *
from models import *
from scheduler import *
from attention_maps import *
from helpers import *

# Load Data

Select wheter to use old or new data, or both. Default is training with both old and new, testing with new only

In [ ]:
# TRAINING SET
train_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/old_first/train",
              BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/train_new"]    # Old and new
# train_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/train_new"]  # New only

# TESTING SET
# test_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/old_first/val",
#              BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new"]    # Old and new
test_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new"]      # New only

Label encoder

In [ ]:
class LabelEncoder():
    def __init__(self):
        self.encode_map = {
            'Eating': 0,
            'Face touching': 0,
            'Eye rubbing': 1,
            'Eye rubbing light': 1,
            'Eye rubbing moderate': 1,
            'Eye touching': 0,
            'Glasses readjusting': 0,
            'Hair combing': 2,
            'Make up': 0,
            'Make up application': 0,
            'Make up removal': 0,
            'Skin scratching': 2,
            'Teeth brushing': 3,
            'Nothing': 4,
            'no_label': 4
        }
        self.decode_map = {
            0: "Face touching",
            1: "Eye rubbing",
            2: "Hair combing & Skin scratching",
            # 3: "Skin scratching",
            3: "Teeth brushing",
            4: "Nothing"
        }

    def transform(self, labels):
        return [self.encode_map[label] for label in labels]

    def inv_transform(self, labels):
        return [self.decode_map[label] for label in labels]



label_encoder = LabelEncoder()
CLASSES = list(label_encoder.decode_map.values())

FEATURES = ['accelerometerAccelerationX(G)',
            'accelerometerAccelerationY(G)',
            'accelerometerAccelerationZ(G)',
            'motionYaw(rad)',
            'motionRoll(rad)',
            'motionPitch(rad)',
            'motionRotationRateX(rad/s)',
            'motionRotationRateY(rad/s)',
            'motionRotationRateZ(rad/s)',
            'motionUserAccelerationX(G)',
            'motionUserAccelerationY(G)',
            'motionUserAccelerationZ(G)',
            'motionQuaternionX(R)',
            'motionQuaternionY(R)',
            'motionQuaternionZ(R)',
            'motionQuaternionW(R)',
            'motionGravityX(G)',
            'motionGravityY(G)',
            'motionGravityZ(G)'
]

# Data analysis

Brief analysis of data distribution

In [ ]:
label_encoder = LabelEncoder()
stats_train = pd.DataFrame(columns = ['0','1','2','3','4'])
stats_test = pd.DataFrame(columns = ['0','1','2','3','4'])

In [ ]:
train_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/old_first/train",
              BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/train_new"]
test_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new"]

for path in train_path:
  for subpath, subdir, files in os.walk(path):
      for file in glob(os.path.join(subpath, "*.csv")):
          df = pd.read_csv(file)

          try:
            user = df['user'][0]
          except:
            user = df['userName'][0]

          label = df['label'][0]
          label = str(label_encoder.encode_map[label])

          if not (stats_train.index == user).any():
            stats_train.loc[user] = list(np.zeros(stats_train.shape[1]))

          stats_train.loc[user][label] += 1

for path in test_path:
  for subpath, subdir, files in os.walk(path):
      for file in glob(os.path.join(subpath, "*.csv")):
          df = pd.read_csv(file)

          try:
            user = df['user'][0]
          except:
            user = df['userName'][0]

          label = df['label'][0]
          label = str(label_encoder.encode_map[label])

          if not (stats_test.index == user).any():
            stats_test.loc[user] = list(np.zeros(stats_test.shape[1]))

          stats_test.loc[user][label] += 1

In [ ]:
stats_train

In [ ]:
stats_test

# Train Model

### Train baseline models

Train DeepConvLSTM model

In [ ]:
# Set training parameters and load data
num_epochs = 30
bsz = 50
pl.seed_everything(42)
data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, bsz, normalize = True, add_fft = False)

In [ ]:
model = DeepConvLSTM(n_features=len(FEATURES), n_classes=len(CLASSES), learning_rate = 1e-4, warmup = 1, warmup_iters=1e7, datamodule = data_module)

checkpoint_callback = ModelCheckpoint(
    dirpath = BASE_FOLDER + "logs" + model.__class__.__name__,
    filename = "checkpoint",
    save_top_k = 1, verbose=True,
    monitor = "f1", mode="max")

lr_monitor = LearningRateMonitor(logging_interval='step')
logger = TensorBoardLogger(save_dir=BASE_FOLDER, name='logs' + model.__class__.__name__)

trainer = pl.Trainer(
    max_epochs=num_epochs,
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback, lr_monitor],
    accelerator='gpu'
)

trainer.fit(model, data_module)

# Save model arguments
dir = BASE_FOLDER + 'logs' + model.__class__.__name__
dir += '/kwargs.pt'
dir = uniquify(dir)
torch.save((model.kwargs, model.state_dict(), model.fig, model.best_f1), dir)

Load model and visualize results

In [ ]:
kwargs_dir = BASE_FOLDER + 'logsDeepConvLSTM/kwargs.pt'
check_dir = BASE_FOLDER + 'logsDeepConvLSTM/checkpoint.ckpt'

kwargs, _ , confmat, f1= torch.load(kwargs_dir)
model = DeepConvLSTM.load_from_checkpoint(check_dir, **kwargs)

In [ ]:
print(f1)
confmat

Validate for different test set if necessary

In [ ]:
train_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/train_new"]
test_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user52",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user54",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user57",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user58"]

data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, 1, normalize = True, add_fft = False)

trainer = pl.Trainer(
    max_epochs=10,
    devices=1,
    accelerator='gpu'
)
trainer.validate(model=model, ckpt_path=check_dir, verbose=True, datamodule=data_module)

In [ ]:
img_dir = BASE_FOLDER + 'plots/confmat' + model.__class__.__name__ + '.png'
img_dir = uniquify(img_dir)
model.fig.savefig(img_dir)

### Train attention model

In [ ]:
# Set training parameters and load data
num_epochs = 30
bsz = 50
pl.seed_everything(42)
data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, bsz, normalize = True, add_fft = False)

In [ ]:
model = AttentionModel(n_features=len(FEATURES),
                       n_classes=len(CLASSES),
                       datamodule=data_module,
                       learning_rate = 1e-3,
                       seq_length = 150,
                       input_dim=19,
                       model_dim=128,
                       num_heads= 4,
                       num_layers=4,
                       warmup=100,
                       warmup_iters=10000,
                       dropout=0.1)

checkpoint_callback = ModelCheckpoint(
    dirpath = BASE_FOLDER + "logs" + model.__class__.__name__,
    filename = "checkpoint",
    save_top_k = 1, verbose=True,
    monitor = "f1", mode="max")

lr_monitor = LearningRateMonitor(logging_interval='step')

logger = TensorBoardLogger(save_dir=BASE_FOLDER, name='logs' + model.__class__.__name__)

trainer = pl.Trainer(
    max_epochs=num_epochs,
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback, lr_monitor],
    accelerator='gpu'
)

trainer.fit(model, data_module)


# Save model arguments
dir = BASE_FOLDER + 'logs' + model.__class__.__name__
dir += '/kwargs.pt'
dir = uniquify(dir)
torch.save((model.kwargs, model.state_dict(), model.fig, model.best_f1), dir)

Tensorboard logger

In [ ]:
!kill 4935
%reload_ext tensorboard
%tensorboard --logdir logsAttentionModel/version_0 --samples_per_plugin images=30

Load model

In [ ]:
kwargs_dir = BASE_FOLDER + 'logsAttentionModel/kwargs.pt'
check_dir = BASE_FOLDER + 'logsAttentionModel/checkpoint.ckpt'

kwargs, _ , confmat, f1= torch.load(kwargs_dir)
model = AttentionModel.load_from_checkpoint(check_dir, **kwargs)

In [ ]:
print(f1)
confmat

In [ ]:
img_dir = BASE_FOLDER + 'plots/confmat' + model.__class__.__name__ + '.png'
img_dir = uniquify(img_dir)
confmat.savefig(img_dir)

Validate for different test set

In [ ]:
train_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/train_new"]
test_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user52",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user54",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user57",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user58"]

data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, 1, normalize = True, add_fft = False)

trainer = pl.Trainer(
    max_epochs=10,
    devices=1,
    accelerator='gpu'
)

trainer.validate(model=model, ckpt_path=check_dir, verbose=True, datamodule=data_module)

In [ ]:
img_dir = BASE_FOLDER + 'plots/confmat' + model.__class__.__name__ + '.png'
img_dir = uniquify(img_dir)
model.fig.savefig(img_dir)

Attention maps

In [ ]:
train_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/train_new"]
test_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user52",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user54",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user57",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user58"]

data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, 1, normalize = True, add_fft = False)
data_module.setup()

In [ ]:
it = iter(data_module.val_dataloader())
data_input, labels = next(it)
for l in list(label_encoder.decode_map.keys()):
    correct_found = False
    while not correct_found:
        data_input, labels = next(it)
        if labels == torch.tensor(l):
            y_hat = model.forward(data_input)
            y_pred = torch.argmax(y_hat, dim=1)
            if y_pred == l:
                correct_found = True

    attention_maps = model.get_attention_maps(data_input)

    label_str = label_encoder.decode_map[labels.item()]
    filename = BASE_FOLDER + 'plots/attention_maps/' + model.__class__.__name__ + '_' + label_str +'.png'
    plot_attention_maps(data_input, label_str , attention_maps, idx=0, filename=filename)
    it = iter(data_module.val_dataloader())

### Train attention model with LSTM

In [ ]:
# Set training parameters and load data
num_epochs = 40
bsz = 50
pl.seed_everything(42)
data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, bsz, normalize = True, add_fft = False)

In [ ]:
model = LSTM_AttentionModel(n_features=len(FEATURES),
                       n_classes=len(CLASSES),
                       datamodule=data_module,
                       learning_rate = 1e-3,
                       seq_length = 150,
                       input_dim=19,
                       model_dim=128,
                       num_heads=4,
                       lstm_layers = 2,
                       num_layers=4,
                       warmup=100,
                       warmup_iters=10000,
                       dropout=0.1)

checkpoint_callback = ModelCheckpoint(
    dirpath = BASE_FOLDER + "logs" + model.__class__.__name__,
    filename = "checkpoint",
    save_top_k = 1, verbose=True,
    monitor = "f1", mode="max")

lr_monitor = LearningRateMonitor(logging_interval='step')

logger = TensorBoardLogger(save_dir=BASE_FOLDER, name='logs' + model.__class__.__name__)

trainer = pl.Trainer(
    max_epochs=num_epochs,
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback, lr_monitor],
    accelerator='gpu'
)

trainer.fit(model, data_module)


# Save model arguments
dir = BASE_FOLDER + 'logs' + model.__class__.__name__
dir += '/kwargs.pt'
dir = uniquify(dir)
torch.save((model.kwargs, model.state_dict(), model.fig, model.best_f1), dir)

Tensorboard logger

In [ ]:
!kill 83255
%reload_ext tensorboard
%tensorboard --logdir logsLSTM_AttentionModel/best_5lab --samples_per_plugin images=50

Load model

In [ ]:
kwargs_dir = BASE_FOLDER + 'logsLSTM_AttentionModel/best_5lab_kwargs.pt'
check_dir = BASE_FOLDER + 'logsLSTM_AttentionModel/best_5lab.ckpt'

kwargs, _ , confmat, f1 = torch.load(kwargs_dir)
model = LSTM_AttentionModel.load_from_checkpoint(check_dir, **kwargs)

In [ ]:
print(f1)
confmat

Validate on different test set

In [ ]:
train_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/train_new"]
test_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user52",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user54",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user57",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user58"]

data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, 1, normalize = True, add_fft = False)

trainer = pl.Trainer(
    max_epochs=10,
    devices=1,
    accelerator='gpu'
)
trainer.validate(model=model, ckpt_path=check_dir, verbose=True, datamodule=data_module)

In [ ]:
img_dir = BASE_FOLDER + 'plots/confmat' + model.__class__.__name__ + '.png'
img_dir = uniquify(img_dir)
model.fig.savefig(img_dir)

In [ ]:
model.best_f1

Attention maps

In [ ]:
train_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/train_new"]
test_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user52",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user54",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user57",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user58"]


data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, 1, normalize = True, add_fft = False)
data_module.setup()

In [ ]:
it = iter(data_module.val_dataloader())
data_input, labels = next(it)
for l in list(label_encoder.decode_map.keys()):
    correct_found = False
    while not correct_found:
        data_input, labels = next(it)
        if labels == torch.tensor(l):
            y_hat = model.forward(data_input)
            y_pred = torch.argmax(y_hat, dim=1)
            if y_pred == l:
                correct_found = True

    attention_maps = model.get_attention_maps(data_input)

    label_str = label_encoder.decode_map[labels.item()]
    filename = BASE_FOLDER + 'plots/attention_maps/' + model.__class__.__name__ + '_' + label_str +'.png'
    plot_attention_maps(data_input, label_str , attention_maps, idx=0, filename=filename)
    it = iter(data_module.val_dataloader())

# Tune hyperparameters
Uses online resource WandB
Random search of the best hyperparameter set is performed by running training for permutations of preset parameters, and recording F1 scores. Possibility of visualizing plots and results on wandb sweep website, link provided by cells below

In [ ]:
wandb.login()

Define the sweep

In [ ]:
# Choose searching method
sweep_config = {
    'method' : 'random'
}

# Define objective metric
metric = {
    'name' : 'f1',
    'goal' : 'maximize'
}

sweep_config['metric'] = metric

# Set values for hyperparameters
parameters_dict = {
    'n_layers': {
        'values' : [1,2,4]
    },
    'lstm_layers': {
        'values' : [1,2]
    },
    'n_heads': {
        'values' : [2,4]
    },
    'emb_dim': {
        'values' : [64,128,256]
    },
    'lr': {
        'values' : [1e-3]
    },
    'warmup': {
        'values' : [100,200]
    },
    'dropout': {
        'values' : [0.1,0.2]
    }
}

sweep_config['parameters'] = parameters_dict

# Directly set fixed parameters
# parameters_dict.update({
#     'seq_length' : {
#         'value': 50
#     }
# })

# Visualize dictionary
# import pprint
# pprint.pprint(sweep_config)

Initialize the sweep

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = 'Semester')

Run the sweep agent

In [ ]:
def tuning(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        num_epochs = 30

        bsz = 50
        data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, bsz, normalize = True, add_fft = False)

        # # WITHOUT LSTM
        # model = AttentionModel(n_features=len(FEATURES),
        #                        n_classes=len(CLASSES),
        #                        datamodule=data_module,
        #                        learning_rate = config.lr,
        #                        seq_length = 50,
        #                        input_dim=19,
        #                        model_dim=config.emb_dim*config.n_heads,
        #                        num_heads=config.n_heads,
        #                        num_layers=config.n_layers,
        #                        warmup=config.warmup,
        #                        dropout=config.dropout)

        # WITH LSTM
        model = LSTM_AttentionModel(n_features=len(FEATURES),
                              n_classes=len(CLASSES),
                              datamodule=data_module,
                              learning_rate = config.lr,
                              seq_length = 150,
                              input_dim=19,
                              model_dim=config.emb_dim*config.n_heads,
                              num_heads=config.n_heads,
                              lstm_layers = config.lstm_layers,
                              num_layers=config.n_layers,
                              warmup=config.warmup,
                              dropout=config.dropout)

        # WITH GPU
        trainer = pl.Trainer(
            max_epochs=num_epochs,
            devices=1,
            accelerator='gpu'
        )

        trainer.fit(model, data_module)


In [ ]:
# Set training parameters and load data
pl.seed_everything(42)

# UNCOMMENT WANDB LINES IN models.py
wandb.agent(sweep_id, tuning, count=50)

# Transfer learning

In [ ]:
class LabelEncoder():
    def __init__(self):
        self.encode_map = {
            'Eating': 0,
            'Face touching': 0,
            'Eye rubbing': 1,
            'Eye rubbing light': 1,
            'Eye rubbing moderate': 1,
            'Eye touching': 0,
            'Glasses readjusting': 0,
            'Hair combing': 2,
            'Make up': 0,
            'Make up application': 0,
            'Make up removal': 0,
            'Skin scratching': 2,
            'Teeth brushing': 3,
            'Nothing': 4,
            'no_label': 4
        }
        self.decode_map = {
            0: "Face touching",
            1: "Eye rubbing",
            2: "Hair combing & Skin scratching",
            # 3: "Skin scratching",
            3: "Teeth brushing",
            4: "Nothing"
        }

    def transform(self, labels):
        return [self.encode_map[label] for label in labels]

    def inv_transform(self, labels):
        return [self.decode_map[label] for label in labels]



label_encoder = LabelEncoder()
CLASSES = list(label_encoder.decode_map.values())

FEATURES = ['accelerometerAccelerationX(G)',
            'accelerometerAccelerationY(G)',
            'accelerometerAccelerationZ(G)',
            'motionYaw(rad)',
            'motionRoll(rad)',
            'motionPitch(rad)',
            'motionRotationRateX(rad/s)',
            'motionRotationRateY(rad/s)',
            'motionRotationRateZ(rad/s)',
            'motionUserAccelerationX(G)',
            'motionUserAccelerationY(G)',
            'motionUserAccelerationZ(G)',
            'motionQuaternionX(R)',
            'motionQuaternionY(R)',
            'motionQuaternionZ(R)',
            'motionQuaternionW(R)',
            'motionGravityX(G)',
            'motionGravityY(G)',
            'motionGravityZ(G)'
]

Split users originally for testing in training and testing. Ignore "no label" class (too few data points)

In [ ]:
import shutil

user_path = [BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user52",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user54",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user57",
             BASE_FOLDER + "HFI_data/HFI_data/HandFace_data/new/test_new/user58",]

DESTINATION = BASE_FOLDER + "HFI_data/Transfer_data_withNO/"
users = ['user52','user54','user57','user58']

for ind, path in enumerate(user_path):
    user_name = users[ind]
    print(user_name+'...')

    label_counter = np.zeros(len(label_encoder.decode_map)) # !!!!!

    if not os.path.exists(DESTINATION + user_name + "/train/"):
        os.makedirs(os.path.dirname(DESTINATION + user_name + "/train/"), exist_ok=True)
        os.makedirs(os.path.dirname(DESTINATION + user_name + "/test/"), exist_ok=True)

    for subpath, subdir, files in os.walk(path):
        for file in glob(os.path.join(subpath, "*.csv")):
            df = pd.read_csv(file)

            label = df['label'][0]
            label = label_encoder.encode_map[label]

            file_name = os.path.basename(os.path.normpath(file))

            if label_counter[label] < 5 and label != len(label_encoder.decode_map)-1:
                label_counter[label] += 1
                shutil.copyfile(file, DESTINATION + user_name + "/train/" + file_name)
            else:
                shutil.copyfile(file, DESTINATION + user_name + "/test/" + file_name)
    try:
        print(df['user'][0])
    except:
        print(df['userName'][0])

Load best model

In [ ]:
kwargs_dir = BASE_FOLDER + 'logsLSTM_AttentionModel/best_5lab_kwargs.pt'
check_dir = BASE_FOLDER + 'logsLSTM_AttentionModel/best_5lab.ckpt'

kwargs, _ , confmat, f1= torch.load(kwargs_dir)
model = LSTM_AttentionModel.load_from_checkpoint(check_dir, **kwargs)

Evaluate model on user test set

In [ ]:
user = 'user54'

In [ ]:
# Load data
train_path = [BASE_FOLDER + "HFI_data/Transfer_data_withNO/" + user + "/train"]
test_path = [BASE_FOLDER + "HFI_data/Transfer_data_withNO/" + user + "/test"]
bsz = 1
pl.seed_everything(42)
data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, bsz, normalize = True)

In [ ]:
trainer = pl.Trainer(
    max_epochs=10,
    devices=1,
    accelerator='gpu'
)
model.best_f1 = 0
trainer.validate(model=model, ckpt_path=check_dir, verbose=True, datamodule=data_module)

In [ ]:
print(model.best_f1)
model.fig

In [ ]:
img_dir = BASE_FOLDER + 'plots/transfer_learning/confmat' + model.__class__.__name__ + user + '_before.png'
img_dir = uniquify(img_dir)
model.fig.savefig(img_dir)

Train last layer for user (transfer learning)

In [ ]:
# Load model, freeze layers
for ind, param in enumerate(model.parameters()):
    param.requires_grad = False
# Unfreeze last layer
for param in model.classifier.parameters():
    param.requires_grad = True

# Change learning rate parameters
model.learning_rate = 1e-3
model.warmup = 1
model.warmup_iters = 1e6

# Load data
bsz = 1
pl.seed_everything(42)
data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, bsz, normalize = True)
model.datamodule=data_module

# Train
pl.seed_everything(42)
num_epochs = 5

checkpoint_callback = ModelCheckpoint(
    dirpath = BASE_FOLDER + "logs_transfer_" + model.__class__.__name__+ '/'+user,
    filename = "checkpoint",
    save_top_k = 1, verbose=True,
    monitor = "f1", mode="max")

lr_monitor = LearningRateMonitor(logging_interval='step')

logger = TensorBoardLogger(save_dir=BASE_FOLDER, name='logs_transfer_' + model.__class__.__name__+ '/'+user)

trainer = pl.Trainer(
    max_epochs=num_epochs,
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback, lr_monitor],
    accelerator='gpu'
)

trainer.fit(model, data_module)


# Save model arguments
dir = BASE_FOLDER + 'logs_transfer_' + model.__class__.__name__+ '/'+user
dir += '/kwargs.pt'
dir = uniquify(dir)
torch.save((model.kwargs, model.state_dict(), model.fig, model.best_f1), dir)

Evaluate results

In [ ]:
# Load data
train_path = [BASE_FOLDER + "HFI_data/Transfer_data_withNO/" + user + "/train"]
test_path = [BASE_FOLDER + "HFI_data/Transfer_data_withNO/" + user + "/test"]
bsz = 1
pl.seed_everything(42)
data_module = HandFaceDataModule(train_path, test_path, FEATURES, label_encoder, bsz, normalize = True)

In [ ]:
check_dir = BASE_FOLDER + 'logs_transfer_LSTM_AttentionModel/'+user+'/5labwithNO.ckpt'
kwargs_dir = BASE_FOLDER + 'logs_transfer_LSTM_AttentionModel/'+user+'/5labwithNO_kwargs.pt'

kwargs, _ , confmat, f1= torch.load(kwargs_dir)
model = LSTM_AttentionModel.load_from_checkpoint(check_dir, **kwargs)

trainer = pl.Trainer(
    max_epochs=10,
    devices=1,
    accelerator='gpu'
)
model.best_f1 = 0
trainer.validate(model=model, ckpt_path=check_dir, verbose=True, datamodule=data_module)

In [ ]:
print(model.best_f1)
model.fig

In [ ]:
img_dir = BASE_FOLDER + 'plots/transfer_learning/confmat' + model.__class__.__name__ + user + '_after.png'
img_dir = uniquify(img_dir)
model.fig.savefig(img_dir)